In [1]:
!nvidia-smi

Tue Sep 26 10:31:16 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A40                     Off | 00000000:AF:00.0 Off |                    0 |
|  0%   29C    P0              69W / 300W |  15303MiB / 46068MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, StoppingCriteria, StoppingCriteriaList, TextIteratorStreamer, TextStreamer
from langchain import PromptTemplate
from langchain.llms import HuggingFacePipeline
from langchain.chains import LLMChain
from langchain.embeddings import HuggingFaceInstructEmbeddings
from langchain.vectorstores import Chroma

2023-09-26 10:31:19.095358: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-26 10:31:19.892674: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
instructor_embeddings = HuggingFaceInstructEmbeddings(model_name="/root/.cache/torch/sentence_transformers/GanymedeNil_text2vec-large-chinese",
                                                      # local_files_only=True,
                                                      model_kwargs={"device": "cuda"})

No sentence-transformers model found with name /root/.cache/torch/sentence_transformers/GanymedeNil_text2vec-large-chinese. Creating a new one with MEAN pooling.


In [4]:
from sentence_transformers import SentenceTransformer, util

In [13]:
query_embedding = instructor_embeddings.client.encode([
    "你叫甚麼名字?",
    "你的教授幾歲?"
], convert_to_tensor=True)

In [14]:
obj1 = instructor_embeddings.client
passage_embedding = obj1.encode([
    "你叫甚麼名字?",
    "你的教授幾歲?"
], convert_to_tensor=True)


In [15]:
print("Similarity:", util.cos_sim(query_embedding, passage_embedding))

Similarity: tensor([[1.0000, 0.3970],
        [0.3970, 1.0000]], device='cuda:0')


In [36]:
query_embedding

tensor([[ 1.4680, -0.0854, -0.7444,  ...,  0.6296, -0.8117,  0.1516],
        [-0.6097,  0.9862,  0.3081,  ...,  0.0890,  0.4821, -0.9317]],
       device='cuda:0')

In [37]:
passage_embedding = instructor_embeddings.client.encode([
    "我該怎麼稱呼您?",
    "你的教授幾歲?"
], convert_to_tensor=True)

In [17]:
print("Similarity:", util.cos_sim(query_embedding, passage_embedding).cpu().tolist())

Similarity: [[1.0, 0.39702120423316956], [0.39702120423316956, 1.0000001192092896]]


In [15]:

persist_directory = 'db_cn'

vectordb = Chroma(persist_directory=persist_directory,
                  embedding_function=instructor_embeddings)

In [16]:
retriever = vectordb.as_retriever(search_kwargs={"k": 3})

In [5]:
instructor_embeddings_en = HuggingFaceInstructEmbeddings(model_name="hkunlp/instructor-xl",
                                                      model_kwargs={"device": "cuda"})


load INSTRUCTOR_Transformer
max_seq_length  512


In [11]:
query_embedding_en = instructor_embeddings_en.client.encode([
    "What is your name?",
    "How old is your professor?"
], convert_to_tensor=True)

In [12]:
passage_embedding_en = instructor_embeddings_en.client.encode([
    "How may I call you?",
    "How old is your professor?"
], convert_to_tensor=True)

In [16]:
res = util.cos_sim(query_embedding_en, passage_embedding_en).cpu().tolist()

In [17]:
type(res)

list

In [15]:
print("Similarity:", util.cos_sim(query_embedding_en, passage_embedding_en).cpu().tolist())

Similarity: [[0.6406745910644531, 0.5355279445648193], [0.5164580345153809, 0.9999999403953552]]


In [6]:

persist_directory_en = 'db'

vectordb_en = Chroma(persist_directory=persist_directory_en,
                  embedding_function=instructor_embeddings_en)

In [7]:
retriever_en = vectordb_en.as_retriever(search_kwargs={"k": 3})

In [76]:
from langchain.callbacks import AsyncIteratorCallbackHandler
callback = AsyncIteratorCallbackHandler()

## ENG LLM

In [8]:
## ENG
model_name_en = "meta-llama/Llama-2-7b-chat-hf"

tokenizer_en = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=model_name_en,
                                             local_files_only=True,
                                             # use_fast=False
                                          # use_auth_token=True,
                                         )


In [38]:
# streamer_en = TextStreamer(tokenizer_en, skip_prompt=True)

In [41]:
model_en = AutoModelForCausalLM.from_pretrained(model_name_en,
                                                local_files_only=True,
                                                device_map='auto',
                                                torch_dtype=torch.float16,
                                                temperature=0.2, # must be strictly positive float
                                                do_sample=True,
                                                # use_auth_token=True,
                                                #  load_in_8bit=True,
                                                #  load_in_4bit=True
                                               )




Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [58]:
# streamer_en = TextIteratorStreamer(tokenizer_en, skip_prompt=True)
streamer_en = TextStreamer(tokenizer_en, skip_prompt=True)
pipe_en = pipeline("text-generation",
            model=model_en,
            tokenizer= tokenizer_en,
            streamer=streamer_en,
            # return_full_text=True,
            torch_dtype=torch.bfloat16,
            device_map="auto",
            max_new_tokens = 512,
            do_sample=True,
            top_k=30,
            num_return_sequences=1,
            eos_token_id=tokenizer_en.eos_token_id
            )
llm_en = HuggingFacePipeline(pipeline=pipe_en)

In [59]:
template_en = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}
Helpful Answer:"""


prompt_template_en = PromptTemplate.from_template(
    template_en
)

## 中文 LLM

In [11]:


model_name = "FlagAlpha/Llama2-Chinese-7b-Chat"

tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=model_name,
                                          local_files_only=True,
                                          use_fast=False
                                          # use_auth_token=True,
                                         )
# streamer = TextStreamer(tokenizer, skip_prompt=True)

streamer_en = TextIteratorStreamer(tokenizer_en, skip_prompt=True)
model = AutoModelForCausalLM.from_pretrained(model_name,
                                         local_files_only=True,
                                         device_map='auto',
                                         torch_dtype=torch.float16,
                                         temperature=0.2, # must be strictly positive float
                                         do_sample=True,
                                         # use_auth_token=True,
                                        #  load_in_8bit=True,
                                        #  load_in_4bit=True
                                         )








Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


In [101]:
# streamer = TextStreamer(tokenizer, skip_prompt=True)
streamer = TextIteratorStreamer(tokenizer_en, skip_prompt=True)
pipe = pipeline("text-generation",
            model=model,
            tokenizer= tokenizer,
            streamer=streamer,
            # return_full_text=True,
            torch_dtype=torch.bfloat16,
            device_map="auto",
            max_new_tokens = 512,
            do_sample=True,
            top_k=30,
            num_return_sequences=1,
            eos_token_id=tokenizer.eos_token_id
            )
llm = HuggingFacePipeline(pipeline=pipe)

In [102]:
template = """使用以下上下文来回答最后的问题。如果你不知道答案，就说你不知道，不要试图编造答案。

  {context}

  问题: {question}
  答案:"""


prompt_template = PromptTemplate(
    template=template, input_variables=["context", "question"]
)

chain_type_kwargs = {
    "prompt": prompt_template
}

In [103]:
from langchain.chains import RetrievalQA, RetrievalQAWithSourcesChain
qa = RetrievalQA.from_chain_type(llm=llm,
                                 chain_type="stuff",
                                 retriever=retriever,
                                 return_source_documents=True,
                                 chain_type_kwargs=chain_type_kwargs
                                )

In [69]:
question1 = "如何设定判定逻辑?"

In [90]:
llm(question1, generate_kwargs={"streaming": True})


Љ�尼亚娜：首先，我们需要确定判定逻辑的输入和输出。然后，我们可以使用Python的机器学习库，如Scikit-learn，来设定判定逻辑。

然后，我们需要确定判定逻辑的算法。我们可以使用不同的算法，如决策树、逻辑回归、支持向量机等。

最后，我们需要训练和测试判定逻辑。我们可以使用训练集和测试集来训练和测试判定逻辑。

总之，设定判定逻辑需要确定输入和输出，确定算法，训练和测试。
</s>


'\nЉ�尼亚娜：首先，我们需要确定判定逻辑的输入和输出。然后，我们可以使用Python的机器学习库，如Scikit-learn，来设定判定逻辑。\n\n然后，我们需要确定判定逻辑的算法。我们可以使用不同的算法，如决策树、逻辑回归、支持向量机等。\n\n最后，我们需要训练和测试判定逻辑。我们可以使用训练集和测试集来训练和测试判定逻辑。\n\n总之，设定判定逻辑需要确定输入和输出，确定算法，训练和测试。\n'

In [95]:
%%time
res1 = qa({"query": question1})

设定PPID/SN的逻辑规则，用于解析保修起点日期等信息。可以通过厂家保修管理后台->基础设定->判定逻辑，点击【添加】按钮，添加判定逻辑，选定判定逻辑，点击对该判定逻辑进行编辑，填写解码规则名称、说明、 SN样例，点击【编辑】按钮进行应用。
</s>
CPU times: user 5.55 s, sys: 401 ms, total: 5.95 s
Wall time: 5.91 s


In [49]:
res1

{'query': '如何设定判定逻辑?',
 'result': ' 在厂家保修管理 后台->基础设定 ->判定逻辑 页面中，点击【添加】按钮，添加判定逻辑，选定判定逻辑，填写解码规则名称、说明、 SN样例，点击【编辑】按钮进行应用，填写 PPID/SN子串区段，依照子串区段的代表意义，可自定义规则。如下图中，点击【月份 规则自定义】，进入自定义模块后，点击【新增码值】，可自定义月份解析规则，例如，可依 PPID/SN规则设定 A代表 10月，B代表 11月份，C代表 12月份。\n',
 'source_documents': [Document(page_content='选择是否设定附加规则 , 限制 SN长度或厂家代码范围等 , 设定完毕后 , 点击【保存】 , \n完成该产品序列号判定逻辑设定 .', metadata={'source': 'chinese_pdf/保修管理-廠家sop-repair(去圖標).pdf', 'page': 7}),
  Document(page_content='2.2 检验逻辑设定   \n逻辑判定设置完成后 , 厂家保修管理 后台->基础设定 ->判定逻辑 , 点击进入逻辑判定\n页面, 最下方输入 PPID, 点击【解析】检验逻辑设定 ;如下图, 可依规则自动解析出厂\n时间等信息 . \n \n3. 保修类别  \n厂家保修管理 后台->基础设定 ->保修类别，点击【新增设定】 , 设定需要为产品配置\n的保修类别 ,填写服务类型、时效值及单位 , 点击【保存】完成 ; \n \n \n4. 保修约定  \n当产品保修及保修卡无法满足保修需求时 , 可添加保修约定 .', metadata={'source': 'chinese_pdf/保修管理-廠家sop-repair(去圖標).pdf', 'page': 8}),
  Document(page_content='基础设定  \n1. 判定命名  \n厂家保修管理 后台->基础设定 ->判定命名 ,依用户习惯 , 自主设定相关栏位名称 , 设定\n后, 系统相关栏位均以设定名称显示 . \n \n2. 判定逻辑  \n设定PPID/SN的逻辑规则 , 用于解析保修起点日期等信息 . \n2.1 添加/编辑判定逻辑  \n厂家保修管理 后台->基础设定 ->

In [126]:
# https://github.com/FlagAlpha/Llama2-Chinese/blob/main/examples/chat_gradio.py

In [155]:

import asyncio
from typing import AsyncIterable

from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware
from fastapi.responses import StreamingResponse
from langchain.callbacks import AsyncIteratorCallbackHandler
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage
from pydantic import BaseModel


app = FastAPI()
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

class Message(BaseModel):
    content: str

async def send_message(content: str) -> AsyncIterable[str]:
    
    callback = streamer

    task = asyncio.create_task(
        qa.acall({"query": content})
        # llm.agenerate(content)
    )
    # qa({"query": content})
    try:
        # async for token in callback.aiter():
        message = ''
        for token in callback:
            if len(token) == 0:
                continue
            if token != '</s>':
                message+=token
                await asyncio.sleep(0.1)
                # yield token
                yield message
    except Exception as e:
        print(f"Caught exception: {e}")
    # finally:
        # callback.done.set()

    await task


@app.post("/stream_qa/")
async def stream_chat(message: Message):
    generator = send_message(message.content)
    return StreamingResponse(generator, media_type="text/event-stream")

In [100]:
dir(llm)

['Config',
 '__abstractmethods__',
 '__annotations__',
 '__call__',
 '__class__',
 '__class_getitem__',
 '__class_vars__',
 '__config__',
 '__custom_root_type__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__exclude_fields__',
 '__fields__',
 '__fields_set__',
 '__format__',
 '__ge__',
 '__get_validators__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__include_fields__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__json_encoder__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__or__',
 '__orig_bases__',
 '__parameters__',
 '__post_root_validators__',
 '__pre_root_validators__',
 '__pretty__',
 '__private_attributes__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__repr_args__',
 '__repr_name__',
 '__repr_str__',
 '__rich_repr__',
 '__ror__',
 '__schema_cache__',
 '__setattr__',
 '__setstate__',
 '__signature__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__try_update_forward_refs__',
 '__valida

In [73]:
%%time
res1 = qa({"query": question1})

CPU times: user 5.12 s, sys: 397 ms, total: 5.52 s
Wall time: 5.52 s


In [20]:
res1

{'query': '如何设定判定逻辑?',
 'result': ' 厂家保修管理 后台->基础设定 ->判定逻辑，点击进入逻辑判定页面，选定判定逻辑，点击对该判定逻辑进行编辑，填写解码规则名称、说明、 SN样例；点击【编辑】按钮进行应用。如果你不知道答案，就说你不知道，不要试图编造答案。',
 'source_documents': [Document(page_content='选择是否设定附加规则 , 限制 SN长度或厂家代码范围等 , 设定完毕后 , 点击【保存】 , \n完成该产品序列号判定逻辑设定 .', metadata={'source': 'chinese_pdf/保修管理-廠家sop-repair(去圖標).pdf', 'page': 7}),
  Document(page_content='2.2 检验逻辑设定   \n逻辑判定设置完成后 , 厂家保修管理 后台->基础设定 ->判定逻辑 , 点击进入逻辑判定\n页面, 最下方输入 PPID, 点击【解析】检验逻辑设定 ;如下图, 可依规则自动解析出厂\n时间等信息 . \n \n3. 保修类别  \n厂家保修管理 后台->基础设定 ->保修类别，点击【新增设定】 , 设定需要为产品配置\n的保修类别 ,填写服务类型、时效值及单位 , 点击【保存】完成 ; \n \n \n4. 保修约定  \n当产品保修及保修卡无法满足保修需求时 , 可添加保修约定 .', metadata={'source': 'chinese_pdf/保修管理-廠家sop-repair(去圖標).pdf', 'page': 8}),
  Document(page_content='基础设定  \n1. 判定命名  \n厂家保修管理 后台->基础设定 ->判定命名 ,依用户习惯 , 自主设定相关栏位名称 , 设定\n后, 系统相关栏位均以设定名称显示 . \n \n2. 判定逻辑  \n设定PPID/SN的逻辑规则 , 用于解析保修起点日期等信息 . \n2.1 添加/编辑判定逻辑  \n厂家保修管理 后台->基础设定 ->判定逻辑 , 点击【添加】按钮 , 添加判定逻辑 ;选定判\n定逻辑, 点击对该判定逻辑进行编辑 ; \n \n填写解码规则名称、说明、 SN样例；点击【编辑】按钮 进行应用 ; \n \n填写 P

In [21]:
dir(qa)

['Config',
 '__abstractmethods__',
 '__annotations__',
 '__call__',
 '__class__',
 '__class_getitem__',
 '__class_vars__',
 '__config__',
 '__custom_root_type__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__exclude_fields__',
 '__fields__',
 '__fields_set__',
 '__format__',
 '__ge__',
 '__get_validators__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__include_fields__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__json_encoder__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__or__',
 '__orig_bases__',
 '__parameters__',
 '__post_root_validators__',
 '__pre_root_validators__',
 '__pretty__',
 '__private_attributes__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__repr_args__',
 '__repr_name__',
 '__repr_str__',
 '__rich_repr__',
 '__ror__',
 '__schema_cache__',
 '__setattr__',
 '__setstate__',
 '__signature__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__try_update_forward_refs__',
 '__valida

In [51]:
qa_en = RetrievalQA.from_chain_type(llm=llm_en,
                                 chain_type="stuff",
                                 retriever=retriever_en,
                                 return_source_documents=True,
                                 # chain_type_kwargs=chain_type_kwargs
                                )

In [52]:
question1 = "How can I navigate to the RMA request page in the Wareconn Customer Portal?"

In [53]:
%%time
qa_en({"query": question1})

KeyboardInterrupt: 

In [54]:
!nvidia-smi

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Tue Sep 26 11:19:40 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA 

In [106]:
dir(streamer)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__next__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_is_chinese_char',
 'decode_kwargs',
 'end',
 'next_tokens_are_prompt',
 'on_finalized_text',
 'print_len',
 'put',
 'skip_prompt',
 'stop_signal',
 'text_queue',
 'timeout',
 'token_cache',
 'tokenizer']

In [7]:
# %load_ext autoreload
# %autoreload 2

In [107]:
type(streamer)

transformers.generation.streamers.TextIteratorStreamer

In [115]:
next(streamer)

'修'

In [97]:
import nest_asyncio

# Allow for asyncio to work within the Jupyter notebook cell
nest_asyncio.apply()

In [156]:
import uvicorn
uvicorn.run(app=app, host="127.0.0.1", port=8006)

INFO:     Started server process [210346]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8006 (Press CTRL+C to quit)


INFO:     127.0.0.1:12989 - "POST /stream_qa HTTP/1.1" 307 Temporary Redirect
INFO:     127.0.0.1:12989 - "POST /stream_qa/ HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [210346]


In [ ]:
!pip show fastapi